# Data Science Capstone Project - Cambridge Data

## Install Necessary Libraries

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Import Data Set

In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
body )
#connection data removed for security reasons
df_Camb = pd.read_csv(body)
df_Camb.head(15)



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,2114,Cambridge,East Cambridge,42.36860,-71.07740
1,2114,Cambridge,Area 2/MIT,42.35700,-71.09290
2,2114,Cambridge,Welllington-Harrington,42.37016,-71.08849
3,2114,Cambridge,The Port,42.36710,-71.09580
4,2114,Cambridge,Cambridgeport,42.35960,-71.10770
5,2114,Cambridge,Mid-Cambridge,42.37360,-71.10970
6,2114,Cambridge,Riverside,42.39770,-71.11370
7,2114,Cambridge,Agassiz,42.38010,-71.11660
8,2114,Cambridge,Neighborhood Nine,42.38590,-71.13000
9,2114,Cambridge,West Cambridge,42.37740,-71.13590


## Acquire Longitute and Latitude for Cambridge and Create Map

In [3]:
address = 'Cambridge, MA'

geolocator = Nominatim(user_agent="Cambridge_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Cambridge are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Cambridge are 42.3750997, -71.1056157.


In [4]:
map_Camb = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighbourhood in zip(df_Camb['Latitude'], df_Camb['Longitude'], df_Camb['Borough'], df_Camb['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Camb)  
    
map_Camb

## Connect to FourSquare API and pulling venue data

In [ ]:
CLIENT ID= #removed for security
CLIENT SECRETE=#removed for security

In [9]:

neighbourhood_latitude = Camb2_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = Camb2_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = Camb2_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} is {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of East Cambridge is 42.3686, -71.0774.


In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FW3I2WWKTVVRBVCHKYKN4QEU1SGSOZAEV2KPJMPCN5CO0QAF&client_secret=PCREM2YUL53YDCQWFMDJ412M3NIJQANYLN4CWZPZVE52UL4J&v=20180605&ll=42.3686,-71.0774&radius=500&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce0d7d0dd57970e2506c61d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4a5f568af964a52067bf1fe3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1fa931735',
         'name': 'Hotel',
         'pluralName': 'Hotels',
         'primary': True,
         'shortName': 'Hotel'}],
       'id': '4a5f568af964a52067bf1fe3',
       'location': {'address': '25 Edwin H Land Blvd',
        'cc': 'US',
        'city': 'Cambridge',
        'country': 'United States',
        'distance': 194,
        'formattedAddress': ['25 Edwin H Land Blvd',
         'Cambridge, MA 02141',
         'United States'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 42.367

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Kimpton Marlowe Hotel,Hotel,42.367964,-71.075195
1,Apple CambridgeSide,Electronics Store,42.367982,-71.075822
2,SEPHORA,Cosmetics Shop,42.367708,-71.076122
3,Helmand Restaurant,Middle Eastern Restaurant,42.366529,-71.078079
4,The Cheesecake Factory,American Restaurant,42.367322,-71.076285


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
print(Camb_venues.shape)
Camb_venues.head()

(337, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Cambridge,42.3686,-71.0774,Kimpton Marlowe Hotel,42.367964,-71.075195,Hotel
1,East Cambridge,42.3686,-71.0774,Apple CambridgeSide,42.367982,-71.075822,Electronics Store
2,East Cambridge,42.3686,-71.0774,SEPHORA,42.367708,-71.076122,Cosmetics Shop
3,East Cambridge,42.3686,-71.0774,Helmand Restaurant,42.366529,-71.078079,Middle Eastern Restaurant
4,East Cambridge,42.3686,-71.0774,The Cheesecake Factory,42.367322,-71.076285,American Restaurant


## Merging Data Sets

In [18]:
# one hot encoding
Camb_onehot = pd.get_dummies(Camb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Camb_onehot['Neighborhood'] = Camb_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Camb_onehot.columns[-1]] + list(Camb_onehot.columns[:-1])
Camb_onehot = Camb_onehot[fixed_columns]

Camb_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Bistro,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Academic Building,College Engineering Building,College Gym,College Technology Building,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Farmers Market,Fish Market,Flower Shop,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hobby Shop,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Museum,Music Venue,New American Restaurant,North Indian Restaurant,Other Great Outdoors,Park,Parking,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool Hall,Portuguese Restaurant,Pub,Public Art,Record Shop,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,East Cambridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,East Cambridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,East Cambridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,East Cambridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,East Cambridge,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
Camb_grouped = Camb_onehot.groupby('Neighborhood').mean().reset_index()
Camb_grouped

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Bistro,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Academic Building,College Engineering Building,College Gym,College Technology Building,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Farmers Market,Fish Market,Flower Shop,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hobby Shop,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Museum,Music Venue,New American Restaurant,North Indian Restaurant,Other Great Outdoors,Park,Parking,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool Hall,Portuguese Restaurant,Pub,Public Art,Record Shop,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Agassiz,0.030303,0.000000,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.060606,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.060606,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.0,0.030303,0.000000,0.000000,0.000000,0.030303,0.030303,0.000000,0.060606,0.000000,0.000000,0.060606,0.000000,0.030303,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.000000
1,Area 2/MIT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0

## Pulling top 15 venues per Neighborhood

In [20]:
num_top_venues = 15

for hood in Camb_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Camb_grouped[Camb_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agassiz----
                          venue  freq
0                Ice Cream Shop  0.06
1                           Pub  0.06
2                        Bakery  0.06
3                    Poke Place  0.06
4            Chinese Restaurant  0.06
5           American Restaurant  0.03
6                 Deli / Bodega  0.03
7                Science Museum  0.03
8                Sandwich Place  0.03
9   College Technology Building  0.03
10                  Record Shop  0.03
11                   Playground  0.03
12                  Pizza Place  0.03
13                         Park  0.03
14               Farmers Market  0.03


----Area 2/MIT----
                           venue  freq
0                Harbor / Marina  0.07
1                        Stadium  0.07
2                   Concert Hall  0.07
3                            Pub  0.07
4                     Public Art  0.07
5                    College Gym  0.07
6   College Engineering Building  0.07
7                    Coffee Shop  0.07
8   

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Camb_grouped['Neighborhood']

for ind in np.arange(Camb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Camb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Agassiz,Chinese Restaurant,Pub,Ice Cream Shop,Poke Place,Bakery,Coffee Shop,Record Shop,Playground,Pizza Place,Park,Museum,Science Museum,College Technology Building,Liquor Store,Deli / Bodega
1,Area 2/MIT,Gym / Fitness Center,Stadium,Harbor / Marina,Food Truck,Donut Shop,Concert Hall,College Gym,College Engineering Building,Coffee Shop,Pub,Public Art,Burrito Place,Snack Place,Pizza Place,Tennis Court
2,Cambridge Highlands,Liquor Store,Pharmacy,Sushi Restaurant,Grocery Store,Hotel,Italian Restaurant,Donut Shop,Lake,Mexican Restaurant,Mobile Phone Shop,Coffee Shop,Bank,Gas Station,Bakery,Theater
3,Cambridgeport,Park,Pizza Place,Dog Run,Food Truck,Museum,Playground,Plaza,Clothing Store,Sandwich Place,Southern / Soul Food Restaurant,Indian Restaurant,Music Venue,Art Gallery,Bakery,Trail
4,East Cambridge,Pizza Place,Gym / Fitness Center,Science Museum,Café,Clothing Store,Cosmetics Shop,Hotel,Lingerie Store,American Restaurant,Chinese Restaurant,Sandwich Place,Park,Parking,Ice Cream Shop,Burrito Place


## Creating Kmeans Algorithm and clusters

In [24]:
# set number of clusters
kclusters = 5

Camb_grouped_clustering = Camb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Camb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
# add clustering labels
kmeans.labels_[0:10] 

array([1, 4, 1, 2, 1, 1, 1, 1, 1, 3], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Camb_merged = Camb2_data


Camb_merged = Camb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Camb_merged.head(15) 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,2114,Cambridge,East Cambridge,42.36860,-71.07740,1,Pizza Place,Gym / Fitness Center,Science Museum,Café,Clothing Store,Cosmetics Shop,Hotel,Lingerie Store,American Restaurant,Chinese Restaurant,Sandwich Place,Park,Parking,Ice Cream Shop,Burrito Place
1,2114,Cambridge,Area 2/MIT,42.35700,-71.09290,4,Gym / Fitness Center,Stadium,Harbor / Marina,Food Truck,Donut Shop,Concert Hall,College Gym,College Engineering Building,Coffee Shop,Pub,Public Art,Burrito Place,Snack Place,Pizza Place,Tennis Court
2,2114,Cambridge,Welllington-Harrington,42.37016,-71.08849,1,Café,Coffee Shop,Fish Market,American Restaurant,Portuguese Restaurant,Seafood Restaurant,Comfort Food Restaurant,Sandwich Place,Salon / Barbershop,Discount Store,Italian Restaurant,Pool Hall,Shopping Mall,Korean Restaurant,Chinese Restaurant
3,2114,Cambridge,The Port,42.36710,-71.09580,1,Plaza,Italian Restaurant,Gym,Coffee Shop,Brewery,Park,New American Restaurant,Café,American Restaurant,Cycle Studio,Chinese Restaurant,Pizza Place,Movie Theater,Mediterranean Restaurant,Lounge
4,2114,Cambridge,Cambridgeport,42.35960,-71.10770,2,Park,Pizza Place,Dog Run,Food Truck,Museum,Playground,Plaza,Clothing Store,Sandwich Place,Southern / Soul Food Restaurant,Indian Restaurant,Music Venue,Art Gallery,Bakery,Trail
5,2114,Cambridge,Mid-Cambridge,42.37360,-71.10970,1,Pizza Place,Yoga Studio,Sandwich Place,Art Museum,Grocery Store,Coffee Shop,Hotel,Park,Cycle Studio,Chinese Restaurant,Pharmacy,College Academic Building,New American Restaurant,College Theater,Diner
6,2114,Cambridge,Riverside,42.39770,-71.11370,1,Breakfast Spot,Vegetarian / Vegan Restaurant,Park,Playground,Ice Cream Shop,Sandwich Place,Indian Restaurant,Italian Restaurant,Laundry Service,Plaza,Coffee Shop,Bowling Alley,Liquor Store,Convenience Store,North Indian Restaurant
7,2114,Cambridge,Agassiz,42.38010,-71.11660,1,Chinese Restaurant,Pub,Ice Cream Shop,Poke Place,Bakery,Coffee Shop,Record Shop,Playground,Pizza Place,Park,Museum,Science Museum,College Technology Building,Liquor Store,Deli / Bodega
8,2114,Cambridge,Neighborhood Nine,42.38590,-71.13000,1,Italian Restaurant,Yoga Studio,Plaza,Bakery,Chinese Restaurant,Dive Bar,Flower Shop,Furniture / Home Store,Grocery Store,Park,Pet Store,Pizza Place,American Restaurant,Sushi Restaurant,Stadium
9,2114,Cambridge,West Cambridge,42.37740,-71.13590,0,American Restaurant,Boutique,Italian Restaurant,Food Truck,Fish Market,Pet Store,Cheese Shop,Bus Stop,Bus Line,Grocery Store,BBQ Joint,Bank,Flower Shop,Dance Studio,Deli / Bodega


## Creating Cluster Map of Cambridge Neighborhoods

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Camb_merged['Latitude'], Camb_merged['Longitude'], Camb_merged['Neighbourhood'], Camb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Showing each Clusters neighborhoods and venues

In [27]:
Camb_merged.loc[Camb_merged['Cluster Labels'] == 0, Camb_merged.columns[[2] + list(range(5, Camb_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
9,West Cambridge,0,American Restaurant,Boutique,Italian Restaurant,Food Truck,Fish Market,Pet Store,Cheese Shop,Bus Stop,Bus Line,Grocery Store,BBQ Joint,Bank,Flower Shop,Dance Studio,Deli / Bodega


In [28]:
Camb_merged.loc[Camb_merged['Cluster Labels'] == 1, Camb_merged.columns[[2] + list(range(5, Camb_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,East Cambridge,1,Pizza Place,Gym / Fitness Center,Science Museum,Café,Clothing Store,Cosmetics Shop,Hotel,Lingerie Store,American Restaurant,Chinese Restaurant,Sandwich Place,Park,Parking,Ice Cream Shop,Burrito Place
2,Welllington-Harrington,1,Café,Coffee Shop,Fish Market,American Restaurant,Portuguese Restaurant,Seafood Restaurant,Comfort Food Restaurant,Sandwich Place,Salon / Barbershop,Discount Store,Italian Restaurant,Pool Hall,Shopping Mall,Korean Restaurant,Chinese Restaurant
3,The Port,1,Plaza,Italian Restaurant,Gym,Coffee Shop,Brewery,Park,New American Restaurant,Café,American Restaurant,Cycle Studio,Chinese Restaurant,Pizza Place,Movie Theater,Mediterranean Restaurant,Lounge
5,Mid-Cambridge,1,Pizza Place,Yoga Studio,Sandwich Place,Art Museum,Grocery Store,Coffee Shop,Hotel,Park,Cycle Studio,Chinese Restaurant,Pharmacy,College Academic Building,New American Restaurant,College Theater,Diner
6,Riverside,1,Breakfast Spot,Vegetarian / Vegan Restaurant,Park,Playground,Ice Cream Shop,Sandwich Place,Indian Restaurant,Italian Restaurant,Laundry Service,Plaza,Coffee Shop,Bowling Alley,Liquor Store,Convenience Store,North Indian Restaurant
7,Agassiz,1,Chinese Restaurant,Pub,Ice Cream Shop,Poke Place,Bakery,Coffee Shop,Record Shop,Playground,Pizza Place,Park,Museum,Science Museum,College Technology Building,Liquor Store,Deli / Bodega
8,Neighborhood Nine,1,Italian Restaurant,Yoga Studio,Plaza,Bakery,Chinese Restaurant,Dive Bar,Flower Shop,Furniture / Home Store,Grocery Store,Park,Pet Store,Pizza Place,American Restaurant,Sushi Restaurant,Stadium
10,North Cambridge,1,Pizza Place,Park,Donut Shop,Baseball Field,Playground,Dance Studio,Movie Theater,Café,Liquor Store,Italian Restaurant,Bistro,Grocery Store,Bar,Garden Center,Home Service
11,Cambridge Highlands,1,Liquor Store,Pharmacy,Sushi Restaurant,Grocery Store,Hotel,Italian Restaurant,Donut Shop,Lake,Mexican Restaurant,Mobile Phone Shop,Coffee Shop,Bank,Gas Station,Bakery,Theater


In [29]:
Camb_merged.loc[Camb_merged['Cluster Labels'] == 2, Camb_merged.columns[[2] + list(range(5, Camb_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
4,Cambridgeport,2,Park,Pizza Place,Dog Run,Food Truck,Museum,Playground,Plaza,Clothing Store,Sandwich Place,Southern / Soul Food Restaurant,Indian Restaurant,Music Venue,Art Gallery,Bakery,Trail


In [30]:
Camb_merged.loc[Camb_merged['Cluster Labels'] == 3, Camb_merged.columns[[2] + list(range(5, Camb_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
12,Strawberry Hill,3,Gym / Fitness Center,Music Venue,Golf Course,Park,Other Great Outdoors,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cosmetics Shop,Cycle Studio,Dive Bar,Convenience Store,Concert Hall


In [31]:
Camb_merged.loc[Camb_merged['Cluster Labels'] == 4, Camb_merged.columns[[2] + list(range(5, Camb_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,Area 2/MIT,4,Gym / Fitness Center,Stadium,Harbor / Marina,Food Truck,Donut Shop,Concert Hall,College Gym,College Engineering Building,Coffee Shop,Pub,Public Art,Burrito Place,Snack Place,Pizza Place,Tennis Court
